# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv file
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hobart,20,AU,1659931831,69,-42.8794,147.3294,52.27,5.75
1,Souillac,9,MU,1659931861,78,-20.5167,57.5167,69.91,7.99
2,Hong Kong,60,HK,1659931842,89,22.2855,114.1577,84.38,5.99
3,Kanlagay,91,PH,1659931861,66,5.8757,121.2915,84.29,4.43
4,Vaini,20,TO,1659931861,94,-21.2000,-175.2000,78.96,9.22
...,...,...,...,...,...,...,...,...,...
562,Vila Velha,20,BR,1659932039,88,-20.3297,-40.2925,75.15,12.66
563,Doha,0,QA,1659931904,75,25.2867,51.5333,93.51,4.61
564,Golden,75,US,1659931801,91,39.7555,-105.2211,66.15,7.00
565,Aklavik,75,CA,1659932040,65,68.2191,-135.0107,56.79,7.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap and add heat layer
fig = gmaps.figure(center=(48.0, -6.0), zoom_level=1)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with the following conditions: wind speed less than 10 mph, cloudiness= 0, and maximum temperature between 70 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
18,Bethel,0,US,1659931865,89,41.3712,-73.4140,79.48,0.00
22,Jamestown,0,US,1659931774,68,42.0970,-79.2353,76.59,8.05
52,Albany,0,US,1659931692,88,42.6001,-73.9662,77.41,3.04
101,Isla Vista,0,US,1659931893,66,34.4133,-119.8610,79.99,6.91
149,Ponta do Sol,0,PT,1659931910,83,32.6667,-17.1000,71.71,3.69


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
 
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
18,Bethel,US,41.3712,-73.4140,
22,Jamestown,US,42.0970,-79.2353,
52,Albany,US,42.6001,-73.9662,
101,Isla Vista,US,34.4133,-119.8610,
149,Ponta do Sol,PT,32.6667,-17.1000,


In [8]:
# Set parameters to find hotels with 5000 meters

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

parameters = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # find city name, lat, lnt from hetel_df dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to parameters
    parameters["location"] = f"{lat},{lng}"

    # url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=parameters).json()
    
    # show the results
    results = response['results']
    
    # save hotel names to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # Show missing results
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("---------")
    
print("---End of Search----")

Retrieving Results for Index 18: Bethel.
Closest hotel in Bethel is Hampton Inn Danbury.
---------
Retrieving Results for Index 22: Jamestown.
Closest hotel in Jamestown is DoubleTree by Hilton Jamestown.
---------
Retrieving Results for Index 52: Albany.
Missing field/result... skipping.
---------
Retrieving Results for Index 101: Isla Vista.
Closest hotel in Isla Vista is Courtyard by Marriott Santa Barbara Goleta.
---------
Retrieving Results for Index 149: Ponta do Sol.
Closest hotel in Ponta do Sol is 1905 Zinos Palace.
---------
Retrieving Results for Index 152: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
---------
Retrieving Results for Index 189: Miepherkeim.
Closest hotel in Miepherkeim is Silvan Grand Otel.
---------
Retrieving Results for Index 203: West Wendover.
Closest hotel in West Wendover is Best Western Plus Wendover Inn.
---------
Retrieving Results for Index 221: Tiznit Province.
Missing field/result... skipping.
---------
Retrieving Results for Index 314

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
18,Bethel,US,41.3712,-73.4140,Hampton Inn Danbury
22,Jamestown,US,42.0970,-79.2353,DoubleTree by Hilton Jamestown
52,Albany,US,42.6001,-73.9662,
101,Isla Vista,US,34.4133,-119.8610,Courtyard by Marriott Santa Barbara Goleta
149,Ponta do Sol,PT,32.6667,-17.1000,1905 Zinos Palace
152,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
189,Miepherkeim,TR,38.1419,41.0081,Silvan Grand Otel
203,West Wendover,US,40.7391,-114.0733,Best Western Plus Wendover Inn
221,Tiznit Province,MA,29.5833,-9.5000,
314,Haines City,US,28.1142,-81.6179,Rodeway Inn & Suites


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))